In [1]:
#Imports
import librosa
import numpy as np
import Models #Our models
import LoadAndPreprocessDataset


#categories=['yes','no','up','down','left','right','on','off','stop','go']
categories=['yes','no','up','down','left','right','on','off','stop','go','zero','one','two','three','four','five','six','seven','eight','nine','unknown']
nCategories=len(categories)
nTestSamples=1000 #3k, 11k




In [2]:
#Load filenames
train,val,test = LoadAndPreprocessDataset.loadDatasetFilenames(nCategories=nCategories)

#Load TEST file
X,y_test=LoadAndPreprocessDataset.loadBatch(test,batch_size=nTestSamples, nCategories=nCategories)
#Preprocess TEST
#X_test=LoadAndPreprocessDataset.MFCC_DELTA(X,n_mfcc=40)
X_test=LoadAndPreprocessDataset.MFCC(X,n_mfcc=40)
#X_test=LoadAndPreprocessDataset.melspect(X)
#X_test=X

#ADD extra dimension for CNN
X_test = X_test[..., np.newaxis]


print('X_test uses',X_test.shape[0],"of",len(test),"files","and occupies",X_test.nbytes,"bytes")

X_test uses 1000 of 11005 files and occupies 40320000 bytes


In [3]:
#load model

save_name="2020-08-07_12-47_DSConvModel21"
from tensorflow.keras.models import load_model
model = load_model('models/'+save_name+'.h5')

# Test the model on test data
score = model.evaluate(X_test, y_test)
y_pred=np.argmax(model.predict(X_test),1)


#Measure time for every single prediction
import time

s=time.time()
y_pred=np.argmax(model.predict(X_test[:500,:,:]),1)
e=time.time()
print("Entire (ms):",(e-s)*1000)

starts=np.empty((len(X_test),))
ends=np.empty((len(X_test),))
for i in range(len(X_test)):
    a=np.array([X_test[i]])
    starts[i]=time.time()
    a=np.argmax(model.predict(a),1)
    ends[i]=time.time()
average=sum(ends-starts)/len(X_test)
print("Average single prediction time (ms):",average*1000)

# 1 prediction or a batch of 10 requires the same amount of time
# a batch of 100 prediction requires the double of 1
# a batch of 500 prediction requires the 3 times of 1

1000/1000 [==============================] - 3s 3ms/sample - loss: 0.1751 - sparse_categorical_accuracy: 0.9440
Entire (ms): 916.2085056304932
Average single prediction time (ms): 30.87128210067749


In [4]:
#Measure also with preprocessing


#Load filenames
train,val,test = LoadAndPreprocessDataset.loadDatasetFilenames(nCategories=nCategories)

#Load TEST file
X,y_test=LoadAndPreprocessDataset.loadBatch(test,batch_size=nTestSamples, nCategories=nCategories)



#Preprocess TEST
#X_test=LoadAndPreprocessDataset.MFCC_DELTA(X,n_mfcc=40)
#X_test=LoadAndPreprocessDataset.MFCC(X,n_mfcc=40)
#X_test=LoadAndPreprocessDataset.melspect(X)
#X_test=X

#ADD extra dimension for CNN
X_test = X_test[..., np.newaxis]



starts=np.empty((len(X),))
ends=np.empty((len(X),))
for i in range(len(X)):
    a=np.array([X[i]])
    starts[i]=time.time()
    #a=LoadAndPreprocessDataset.MFCC_DELTA(a,n_mfcc=40)
    a=LoadAndPreprocessDataset.MFCC(a,n_mfcc=40)
    #a=LoadAndPreprocessDataset.melspect(a)
    a=a[...,np.newaxis]
       
    
    a=np.argmax(model.predict(a),1)
    ends[i]=time.time()
average=sum(ends-starts)/len(X_test)
print("Average single prediction time (ms):",average*1000)

Average single prediction time (ms): 39.65917706489563
